In [ ]:
from data import data
import pandas as pd
import numpy as np
from analyze import analyze
from plots import scatter, trend, lmh, trend_years
from sklearn import linear_model, preprocessing
import matplotlib.pyplot as plt
from Freedom.freedom import freedom

In [ ]:
analyze(data, 2017)

In [ ]:
scatter(data, 2017, ['GDP per capita (current US$)', 'Population, total'])

In [ ]:
trend(data, ["World", "Brazil", "Mexico"], 'GDP per capita (current US$)', [1963, 2017])

In [ ]:
lmh(data, 2017, 'Bank capital to assets ratio (%)', 2)

In [ ]:
trend_years(data, "Brazil", 'Population, total', [2000, 2005], True)

In [ ]:
trend_years(data, "Brazil", ['GDP per capita (current US$)', 'Population, total'], [2000, 2005], True)

In [ ]:
comparison_filter(data)